<a href="https://colab.research.google.com/github/a2r4vind/Machine-Learning-Models/blob/main/Implementing_NeuralNetwork_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf

In [12]:
class NaiveDense:
  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    #creating a matrix, W, of shape (input_size, output_size), initialized with random values
    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval= 1e-1)
    self.W = tf.Variable(w_initial_value)

    # creating a vector, b, of shape (output_size), initialized with zeros
    b_shape = (output_size,)
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)

  #Apply the forward pass.
  def __call__(self, inputs):
      return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self): # method for retrieving layer's weights
      return [self.W, self.b]


In [13]:
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [14]:
model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [15]:
import math

In [16]:
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)
  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels